In [31]:
import numpy as np
import pandas as pd

In [32]:
# ------------------------------------------------
# 1) Helper Function: Parsing Time-Parameterized Vectors
# ------------------------------------------------
def parse_time_vector(vector_string, num_periods):
    """
    Reads in a string vector and returns the corresponding values for each year in a numpy array.

    NOTE: Stepping to or ramping from [old rate] to [new rate] AFTER [duration periods].
    """
    tokens = vector_string.strip().split()
    
    # If there's only one token and it's just a number, interpret as constant
    if len(tokens) == 1:
        try:
            val = float(tokens[0])
            return np.full(num_periods, val)
        except ValueError:
            raise ValueError(f"Could not parse constant rate from '{vector_string}'")
    
    try:
        current_rate = float(tokens[0])
    except ValueError:
        raise ValueError(f"Could not parse the initial rate from '{tokens[0]}'")
    
    year_rates = np.full(num_periods, current_rate)
    current_year = 1  # 1-based index
    idx = 1
    
    while idx < len(tokens):
        duration_and_type = tokens[idx]
        idx += 1
        new_rate_str = tokens[idx]
        idx += 1
        
        mode = duration_and_type[-1].upper()  # 'S' or 'R'
        duration_str = duration_and_type[:-1]
        
        try:
            duration = int(duration_str)
        except ValueError:
            raise ValueError(f"Could not parse duration from '{duration_str}'")
        
        try:
            new_rate = float(new_rate_str)
        except ValueError:
            raise ValueError(f"Could not parse new rate from '{new_rate_str}'")
        
        start_year = current_year
        end_year = current_year + duration
        if end_year > num_periods:
            end_year = num_periods
        
        if mode == 'S':
            for y in range(start_year, end_year):
                if y-1 < num_periods:
                    year_rates[y-1] = current_rate
            step_year_index = end_year - 1
            if 0 <= step_year_index < num_periods:
                year_rates[step_year_index] = new_rate
            current_rate = new_rate
            current_year = end_year
        
        elif mode == 'R':
            # Ramp linearly
            rate_diff = new_rate - current_rate
            for i_year in range(duration + 1):
                frac = i_year / float(duration) if duration > 1 else 1.0
                actual_rate = current_rate + frac * rate_diff
                actual_year = start_year + i_year
                if actual_year <= num_periods:
                    year_rates[actual_year - 1] = actual_rate
            current_rate = new_rate
            current_year = end_year
        
        else:
            raise ValueError(f"Unknown mode '{mode}' in parse_time_vector.")
    
    for y in range(current_year, num_periods+1):
        if y-1 < num_periods:
            year_rates[y-1] = current_rate
    
    return year_rates


# ------------------------------------------------
# 2) Helper Function: Draw from Chosen Distribution
# ------------------------------------------------
def draw_random(dist_type, mean, stdev, size=1):
    """
    dist_type: 'normal' or 'lognormal'.
    mean, stdev refer to the *arithmetic* mean and std. dev (if lognormal).
    """
    dist_type = dist_type.lower()
    if dist_type == 'normal':
        return np.random.normal(loc=mean, scale=stdev, size=size)
    elif dist_type == 'lognormal':
        if mean <= 0:
            return np.zeros(size)
        sigma_sq = stdev**2
        mu_sq = mean**2
        phi = np.sqrt(np.log(1 + sigma_sq/mu_sq))   
        m   = np.log(mu_sq / np.sqrt(sigma_sq+mu_sq))
        return np.random.lognormal(mean=m, sigma=phi, size=size)
    else:
        raise ValueError(f"Unsupported distribution type '{dist_type}'")


# ------------------------------------------------
# 3) Main Simulation Function
# ------------------------------------------------
def monte_carlo_retirement(
    num_sims,
    num_periods,
    # Inflation vectors
    inflation_rate_vector,           
    inflation_vol_vector,            
    inflation_dist='normal',
    
    # Asset returns
    asset_mean_vectors=None,
    asset_vol_vectors=None,
    asset_dist_types=None,  # e.g. ['normal','normal','normal',...], length=10
    
    # Allocations
    allocation_vectors=None,  # length=10
    
    # Contribution parameters
    initial_contribution=10000.0,
    contrib_growth_vector="2",
    contrib_vol_vector="1",
    contrib_dist='normal',
    peg_contribution_to_inflation=True, #if true, then contribution growth value will be added to inflation value to get total contribution growth
    contribution_stop_year=30, #no contributions from this year and onward
    
    # Drawdown parameters
    initial_drawdown=0.0,
    drawdown_growth_vector="0",
    drawdown_vol_vector="0",
    drawdown_dist='normal',
    peg_drawdown_to_inflation=True, #if true, then drawdown growth value will be added to inflation value to get total drawdown growth
    drawdown_start_year=30, #drawdowns starting this year and onward      
    
    # Initial balance
    initial_balance=0.0,
    
    # For reproducibility
    random_seed=None
):
    """
    Returns a DataFrame with both nominal and real (inflation-adjusted) statistics by year.

    Order of operations for a given period:
    Period pre-growth balance = last period final balance + contribution
    Period post-growth balance = Period pre-growth balance * (1 + weighted average growth rate)
    Period final balance = Period post-growth balance - drawdown

    Contributions happen at the beginning of the period, then inflation, then investment growth, then drawdowns.     
    Therefore, contributions are inflation adjusted up through the previous period's cumulative inflation. 
    Drawdowns are inflation adjusted up through the current period's cumulative inflation.
    """
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # 1) Parse time-varying vectors
    inflation_mean_array = parse_time_vector(inflation_rate_vector, num_periods)
    inflation_stdev_array = parse_time_vector(inflation_vol_vector, num_periods)
    
    if asset_mean_vectors is None or asset_vol_vectors is None or asset_dist_types is None:
        raise ValueError("Must provide asset_mean_vectors, asset_vol_vectors, asset_dist_types (all length=10).")
    if len(asset_mean_vectors) != 10 or len(asset_vol_vectors) != 10 or len(asset_dist_types) != 10:
        raise ValueError("asset_* parameters must each be length=10.")
    
    asset_mean_arrays = [parse_time_vector(v, num_periods) for v in asset_mean_vectors]
    asset_stdev_arrays = [parse_time_vector(v, num_periods) for v in asset_vol_vectors]
    
    if allocation_vectors is None or len(allocation_vectors) != 10:
        raise ValueError("Must provide 10 allocation_vectors (one per asset).")
    allocation_arrays = [parse_time_vector(v, num_periods) for v in allocation_vectors]
    
    contrib_growth_array = parse_time_vector(contrib_growth_vector, num_periods)
    contrib_stdev_array = parse_time_vector(contrib_vol_vector, num_periods)
    
    drawdown_growth_array = parse_time_vector(drawdown_growth_vector, num_periods)
    drawdown_stdev_array = parse_time_vector(drawdown_vol_vector, num_periods)
    
    # 2) Prepare arrays to store results
    all_balances = np.zeros((num_sims, num_periods))
    all_inflations = np.zeros((num_sims, num_periods))
    all_contributions = np.zeros((num_sims, num_periods))
    all_drawdowns = np.zeros((num_sims, num_periods))
    all_effective_returns = np.zeros((num_sims, num_periods))
    
    all_real_balances = np.zeros((num_sims, num_periods))
    all_real_contributions = np.zeros((num_sims, num_periods))
    all_real_drawdowns = np.zeros((num_sims, num_periods))
    
    all_cum_inflation = np.ones((num_sims, num_periods))
    
    # 3) Run simulations
    for s in range(num_sims):
        balance = initial_balance
        last_contribution = initial_contribution
        cum_infl = 1.0
        
        for t in range(num_periods):
            # 3.1 Draw inflation
            infl_mean_t = inflation_mean_array[t] / 100.0
            infl_std_t  = inflation_stdev_array[t] / 100.0
            this_infl = draw_random(inflation_dist, infl_mean_t, infl_std_t, size=1)[0]
            cum_infl *= (1.0 + this_infl)
            all_cum_inflation[s, t] = cum_infl
            
            # 3.2 Draw returns for the 10 assets
            asset_returns_t = np.zeros(10)
            for i in range(10):
                mean_i = asset_mean_arrays[i][t] / 100.0
                std_i  = asset_stdev_arrays[i][t] / 100.0
                dist_i = asset_dist_types[i]
                asset_returns_t[i] = draw_random(dist_i, mean_i, std_i, size=1)[0]
            
            # 3.3 Contribution logic
            if t < contribution_stop_year:
                c_growth_mean_t = contrib_growth_array[t] / 100.0
                c_growth_std_t  = contrib_stdev_array[t] / 100.0
                c_growth_draw   = draw_random(contrib_dist, c_growth_mean_t, c_growth_std_t, size=1)[0]
                
                if peg_contribution_to_inflation:
                    c_growth = this_infl + c_growth_draw
                else:
                    c_growth = c_growth_draw
                
                if t == 0:
                    this_contribution = last_contribution
                else:
                    this_contribution = all_contributions[s, t-1] * (1 + c_growth)
            else:
                # After the stop year, no further contributions
                this_contribution = 0.0
            
            balance += this_contribution
            
            # 3.4 Weighted return
            weighted_return = 0.0
            for i in range(10):
                alloc_pct = allocation_arrays[i][t] / 100.0
                weighted_return += alloc_pct * asset_returns_t[i]
            balance *= (1.0 + weighted_return)
            
            # 3.5 Drawdown logic
            if t < drawdown_start_year:
                this_drawdown = 0.0
            elif t == drawdown_start_year:
                this_drawdown = initial_drawdown
            else:
                d_growth_mean_t = drawdown_growth_array[t] / 100.0
                d_growth_std_t  = drawdown_stdev_array[t] / 100.0
                d_growth_draw   = draw_random(drawdown_dist, d_growth_mean_t, d_growth_std_t, size=1)[0]
                
                if peg_drawdown_to_inflation:
                    d_growth = this_infl + d_growth_draw
                else:
                    d_growth = d_growth_draw

                this_drawdown = all_drawdowns[s, t-1] * (1 + d_growth)
            
            balance -= this_drawdown
            if balance < 0:
                balance = 0.0
            
            # 3.6 Store results
            all_balances[s, t] = balance
            all_inflations[s, t] = this_infl
            all_contributions[s, t] = this_contribution
            all_drawdowns[s, t] = this_drawdown
            all_effective_returns[s, t] = weighted_return
            
            # Real values
            all_real_balances[s, t]       = balance / cum_infl
            all_real_contributions[s, t]  = this_contribution / (cum_infl /(1+ this_infl))
            all_real_drawdowns[s, t]      = this_drawdown / cum_infl
    
    # 4) Summaries
    percentiles = [10, 25, 50, 75, 90]
    output_dict = {'Year': list(range(1, num_periods+1))}
    
    # Nominal balance percentiles
    for p in percentiles:
        col_name = f"Nominal_Balance_P{p}"
        output_dict[col_name] = [
            np.percentile(all_balances[:, t], p) for t in range(num_periods)
        ]
    
    # Real balance percentiles
    for p in percentiles:
        col_name = f"Real_Balance_P{p}"
        output_dict[col_name] = [
            np.percentile(all_real_balances[:, t], p) for t in range(num_periods)
        ]
    
    # Averages
    output_dict['Avg_Effective_Return'] = [
        np.mean(all_effective_returns[:, t]) * 100.0 for t in range(num_periods)
    ]
    output_dict['Avg_Inflation'] = [
        np.mean(all_inflations[:, t]) * 100.0 for t in range(num_periods)
    ]
    output_dict['Avg_Nominal_Contribution'] = [
        np.mean(all_contributions[:, t]) for t in range(num_periods)
    ]
    output_dict['Avg_Real_Contribution'] = [
        np.mean(all_real_contributions[:, t]) for t in range(num_periods)
    ]
    output_dict['Avg_Nominal_Drawdown'] = [
        np.mean(all_drawdowns[:, t]) for t in range(num_periods)
    ]
    output_dict['Avg_Real_Drawdown'] = [
        np.mean(all_real_drawdowns[:, t]) for t in range(num_periods)
    ]
    
    for i in range(10):
        output_dict[f"Alloc_Asset{i+1}"] = allocation_arrays[i]
    
    df_results = pd.DataFrame(output_dict)
    return df_results


In [35]:
num_sims = 1000
num_periods = 80

asset_mean_vecs = [
    "8", "6", "5", "7", "9", "4", "3", "5 10S 7", "10", "2"
]
asset_vol_vecs = [
    "15", "12", "10", "13", "18", "8", "5", "10 10S 12", "20", "3"
]
asset_dist_types = ["normal"] * 10  

allocation_vecs = [
    "100",   
    "0",   
    "0",   
    "0",   
    "0",   
    "0",    
    "0",    
    "0",    
    "0",    
    "0",    
]

df_results = monte_carlo_retirement(
    num_sims=num_sims,
    num_periods=num_periods,
    inflation_rate_vector="0 12S 12 12R 24",   
    inflation_vol_vector="0",                
    inflation_dist='normal',
    
    asset_mean_vectors=asset_mean_vecs,
    asset_vol_vectors=asset_vol_vecs,
    asset_dist_types=asset_dist_types,
    allocation_vectors=allocation_vecs,
    
    initial_contribution=10000.0,
    contrib_growth_vector="2 50S 0",   
    contrib_vol_vector="0",          
    contrib_dist='normal',
    peg_contribution_to_inflation=False,
    contribution_stop_year=50, 
    
    initial_drawdown=80000.0, 
    drawdown_growth_vector="3",  
    drawdown_vol_vector="0",
    drawdown_dist='normal',
    peg_drawdown_to_inflation=False,
    drawdown_start_year=50, 
    
    initial_balance=0.0,
    random_seed=42
)
df_results.to_excel("output.xlsx")
df_results  


,Year,Nominal_Balance_P10,Nominal_Balance_P25,Nominal_Balance_P50,Nominal_Balance_P75,Nominal_Balance_P90,Real_Balance_P10,Real_Balance_P25,Real_Balance_P50,Real_Balance_P75,...,Alloc_Asset1,Alloc_Asset2,Alloc_Asset3,Alloc_Asset4,Alloc_Asset5,Alloc_Asset6,Alloc_Asset7,Alloc_Asset8,Alloc_Asset9,Alloc_Asset10
0,1,8.754508e+03,9.753695e+03,1.084001e+04,1.179485e+04,1.273071e+04,8754.508496,9753.694986,10840.007115,11794.854779,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1.825620e+04,2.028277e+04,2.272646e+04,2.518109e+04,2.736912e+04,18256.199841,20282.773323,22726.460724,25181.087861,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2.778869e+04,3.125844e+04,3.557784e+04,4.000967e+04,4.391882e+04,27788.692218,31258.436725,35577.836786,40009.669842,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,3.808664e+04,4.324831e+04,4.972112e+04,5.686742e+04,6.380143e+04,38086.636347,43248.313654,49721.122194,56867.418472,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,4.872351e+04,5.602613e+04,6.583230e+04,7.486021e+04,8.392632e+04,48723.508126,56026.131603,65832.296063,74860.205973,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,76,2.854062e+06,9.491844e+06,2.443831e+07,5.767869e+07,1.091612e+08,5.745559,19.108184,49.197166,116.113898,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,77,3.013394e+06,1.021140e+07,2.578917e+07,6.138268e+07,1.153534e+08,4.892187,16.578010,41.868214,99.653600,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,78,2.917435e+06,1.071693e+07,2.727711e+07,6.567213e+07,1.323225e+08,3.819676,14.031233,35.712795,85.981811,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,79,2.875280e+06,1.105481e+07,2.870891e+07,6.960460e+07,1.402186e+08,3.035875,11.672258,30.312415,73.492277,...,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
